# CartesIO Python ROS Client
## Setting up the system
First off, let us import the required python modules with the following command:

In [1]:
from cartesian_interface.pyci_all import *
import numpy as np

If errors occurred, maybe you forgot to include the CartesIO install path in your PYTHONPATH environment variable, 
such as:

```bash
export PYTHONPATH=$PYTHONPATH:/opt/xbot/lib/python2.7/dist-packages
```

In a separate terminal, run the provided example launch file:

```bash
mon launch cartesian_interface coman.launch gui:=true
```
Note: if you did not install rosmon (recommended, `sudo apt-get install ros-kinetic-rosmon`),
you can use the standard roslaunch tool.

On success, you should see the IIT Coman robot in RViz, with a marker on its left hand.

## Retrieve the client object

You can now instantiate the CartesIO Python ROS Client, as follows


In [2]:
cli = pyci.CartesianInterfaceRos()

Some status information can be display by printing the returned object. You should see the roscpp node name where the client is running, plus a list of defined tasks.

In [3]:
print cli

CartesianInterfaceRos running inside ROS node /cartesio_ros_1580477191691385308
Defined tasks: 
 - left_foot
 - right_foot
 - Com
 - left_hand
 - right_hand
 - Postural
 - JointLimits
 - VelocityLimits



The same information can be retrieved programmatically with the `getTaskList` method, returning a list of defined task names.

In [4]:
cli.getTaskList()

[u'left_foot',
 u'right_foot',
 u'Com',
 u'left_hand',
 u'right_hand',
 u'Postural',
 u'JointLimits',
 u'VelocityLimits']

## Interact with a generic task
Let us now focus on one specific task, say `left_hand`, and retrieve some information about it.

In [5]:
task_name = 'left_hand'
larm = cli.getTask(task_name)

We can now get some generic task information such as the type, the name, the lambda value, etc...

In [6]:
print 'Task name is', larm.getName()
print 'Task type is', larm.getType()
print 'Task activation state is', larm.getActivationState()
print 'Task size is', larm.getSize()
print 'Task lambda is', larm.getLambda()
print 'Task active indices are', larm.getIndices()
print 'Task weight is: \n', larm.getWeight()

Task name is left_hand
Task type is Cartesian
Task activation state is ActivationState.Enabled
Task size is 6
Task lambda is 0.10000000149
Task active indices are [0, 1, 2, 3, 4, 5]
Task weight is: 
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


## Cartesian task API
So far, this API is a generic one, applicable to all tasks regardless of their type (base class API).
Since we know that out `right_hand` task is a `Cartesian` task, we can also have access to the `CartesianTask`
specific API. 

In C++ this would require an explicit `dynamic_cast`. In python, we get **automatic down-casting**, as we can immediately check by printing the task Python type:

In [7]:
print type(larm)

<class 'cartesian_interface.pyci.CartesianTaskRos'>


Let us get some `Cartesian`-specific information about the task:

In [8]:
print 'Task distal link is', larm.getDistalLink()
print 'Task base link is', larm.getBaseLink()
print 'Task control mode is', larm.getControlMode()
print 'Task state is', larm.getTaskState()
print 'Task current reference is \n', larm.getPoseReference()[0] # [0] for pose, [1] for velocity


Task distal link is LSoftHand
Task base link is world
Task control mode is ControlType.Position
Task state is State.Online
Task current reference is 
translation: [ 0.1386,  0.1769, 0.01112]
rotation   : [0.0003863,   -0.4566,   -0.2843,     0.843]


### Sending references
Let us now send a target pose to our task. We will tell the hand to go 0.1m forward in 3.0 seconds.

In [9]:
cli.update() # to update the task curren pose reference
Tref, _, _ = larm.getPoseReference() # just return the pose ref, skip vel & acc
print Tref
Tref.translation_ref()[0] += 0.1
time = 3.0
larm.setPoseTarget(Tref, time)

translation: [ 0.1386,  0.1769, 0.01112]
rotation   : [0.0003863,   -0.4566,   -0.2843,     0.843]


True

The robot in Rviz should move accordingly.

In [10]:
larm.getBaseLink()

u'world'